## MNIST Autoencoder mit PyTorch erstellen


##  Import der benötigten Bibliotheken.

In [ ]:
import torch
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch.nn as nn

import matplotlib.pyplot as plt